<a href="https://colab.research.google.com/github/amrios80/Endeporte/blob/main/endeporte_proyecto_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ENDEPORTE_PROYECTO_FINAL

### Objetivo:
Mediante tecnicas de machine learning establecer las variables que mayor relacion tienen con la desercion e identificar los estudiantes de carreras profesionales o tecnologicas de la entidad que pueden desertar o no de sus programas.

### Contexto del negocio
Para la entidad es dificil identificar tempranamente los estudiantes que están en riesgo de abandonar sus carreras, las intervenciones son a menudo menos efectivas porque las señales de riesgo no aparecen hasta que el estudiante ya ha tomado la decisión de desertar. Se espera que mediante modelos de machine learning se identifiquen con anticipacion los estudiantes que pueden llegar a desertar con el fin de brindar a cada estudiante el tipo y nivel de apoyo que necesita para superar sus dificultades.

In [66]:
!pip install scikit-learn

In [67]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)
from sklearn.preprocessing import LabelEncoder # Import LabelEncoder



##**1. Entendimiento y preparación de los datos:**


### Cargue del dataset

In [90]:
demograficos_url="https://github.com/amrios80/Endeporte/raw/refs/heads/main/data/Demograficos.csv.gz"
historicos_url="https://github.com/amrios80/Endeporte/raw/refs/heads/main/data/HistoricoNotas.csv.gz"

demograficos_df = pd.read_csv(demograficos_url, sep=";", compression='gzip')
historicos_df=pd.read_csv(historicos_url, sep=";", compression="gzip")

<ipython-input-90-89934b14d202>:5: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  historicos_df=pd.read_csv(historicos_url, sep=";", compression="gzip")


### Limpieza de datos

* ***Estandarizacion*** : Segun las reglas acordadas con ENDEPORTE, Se excluyen del estudio las ESPECIALIZACIONES:

In [91]:
demograficos_df.sample(3)

,PERIODO_ACAENICO,CODIGO,UNIDAD,CODIGOPROGRAMA,PROGRAMA,SEMESTRE,PROMEDIOSEMESTRE,GENERO,FECHANACIMIENTO,ESTRATO,CIUDADRESIDENCIA,DEPARTAMENTORESICEDENCIA,ESTADOCIVIL,PROMEDIOGENERAL,SITUACION_ACTUAL,CATEGORIA_ACTUAL,SITUACION_DEL_PERIODO,CATEGORIA_DEL_PERIODO,DISCAPACIDAD,TIPO_DISCAPACIDAD,FECHA_MATRICULA,IDCIUDADNACIMIENTO,CIUDADNACIMIENTO,IDDEPARTAMENTONACIMIENTO,DEPARTAMENTONACIMIENTO,SISBEN,NIVELSISBEN,CANTIDAD,PERIODO,EDAD2,EDAD,FACULTAD,NIVEL_FORMACION
54390,2024 - 1,7306172199,ESCUELA NACIONAL DEL DEPORTE,7306,FISIOTERAPIA,10,"4,9",Femenino,21-11-1999,3.0,CALI,VALLE DEL CAUCA,Soltero,"3,8",ACTIVO,CONTINUIDAD ACADEMICA - EGRESADO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,16/01/24,76001.0,CALI,76.0,VALLE DEL CAUCA,0.0,NaN,1,2024-I,25,Edad entre 18 y 25 años,Fac. Salud,Universitrario
32950,2021 - 2,7306201037,ESCUELA NACIONAL DEL DEPORTE,7306,FISIOTERAPIA,3,"3,1",Femenino,01-08-2001,2.0,JAMUNDI,VALLE DEL CAUCA,Soltero,"3,2",BAJO RENDIMIENTO ACADEMICO,ANTIGUO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,26/07/21,76364.0,JAMUNDI,76.0,VALLE DEL CAUCA,0.0,NaN,1,2021-II,20,Edad entre 18 y 25 años,Fac. Salud,Universitrario
30548,2021 - 1,53212201006,ESCUELA NACIONAL DEL DEPORTE,53212,DEPORTE,3,"3,8",Masculino,28-09-2000,2.0,CALI,VALLE DEL CAUCA,Soltero,"3,6",BAJO RENDIMIENTO ACADEMICO,ANTIGUO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,28/01/21,41001.0,NEIVA,41.0,HUILA,0.0,NaN,1,2021-I,21,Edad entre 18 y 25 años,Fac. Deporte,Universitrario


In [92]:
demograficos_df["NIVEL_FORMACION"].value_counts()

,count
NIVEL_FORMACION,
Universitrario,48022
Tecnológico,4994
Especialización,2403


In [93]:
demograficos_df=demograficos_df[(demograficos_df["NIVEL_FORMACION"] != "Especialización")]



*  ***Eliminacion de duplicados*** : La informacion demografica debe estar a nivel de estudiante, segun lo definido con la entidad, se valida que no haya duplicidad a nivel de codigo del estudiante.



In [94]:
total_rows = demograficos_df.shape[0]
total_rows

53016

In [77]:
demograficos_df.loc[demograficos_df.duplicated(subset=["CODIGO"], keep=False)].sample(3)

,PERIODO_ACAENICO,CODIGO,UNIDAD,CODIGOPROGRAMA,PROGRAMA,SEMESTRE,PROMEDIOSEMESTRE,GENERO,FECHANACIMIENTO,ESTRATO,CIUDADRESIDENCIA,DEPARTAMENTORESICEDENCIA,ESTADOCIVIL,PROMEDIOGENERAL,SITUACION_ACTUAL,CATEGORIA_ACTUAL,SITUACION_DEL_PERIODO,CATEGORIA_DEL_PERIODO,DISCAPACIDAD,TIPO_DISCAPACIDAD,FECHA_MATRICULA,IDCIUDADNACIMIENTO,CIUDADNACIMIENTO,IDDEPARTAMENTONACIMIENTO,DEPARTAMENTONACIMIENTO,SISBEN,NIVELSISBEN,CANTIDAD,PERIODO,EDAD2,EDAD,FACULTAD,NIVEL_FORMACION
32210,2021 - 2,102383181086,ESCUELA NACIONAL DEL DEPORTE,102383,NUTRICION Y DIETETICA,7,4,Masculino,18-07-2000,3.0,CALI,VALLE DEL CAUCA,Soltero,"3,7",EXCLUIDO NO RENOVACION DE MATRICULA,CONTINUIDAD ACADEMICA - EGRESADO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,14/07/21,76001.0,CALI,76.0,VALLE DEL CAUCA,0.0,NaN,1,2021-II,21,Edad entre 18 y 25 años,Fac. Salud,Universitrario
41608,2022 - 2,53212191201,ESCUELA NACIONAL DEL DEPORTE,53212,DEPORTE,8,"4,2",Masculino,23-09-1995,1.0,PALMIRA,VALLE DEL CAUCA,Soltero,"4,1",EXCLUIDO NO RENOVACION DE MATRICULA,CONTINUIDAD ACADEMICA - EGRESADO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,18/07/22,17001.0,MANIZALES,17.0,CALDAS,1.0,4.0,1,2022-II,27,Edad entre 26 y 33 años,Fac. Deporte,Universitrario
12576,2018 - 2,90555172082,ESCUELA NACIONAL DEL DEPORTE,90555,ADMINISTRACION DE EMPRESAS,3,4,Masculino,27-09-1996,2.0,CALI,VALLE DEL CAUCA,Soltero,"4,1",EXCLUIDO ASIMILACION DE PENSUM,REINGRESO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,06/08/18,76001.0,CALI,76.0,VALLE DEL CAUCA,0.0,NaN,1,2018-II,22,Edad entre 18 y 25 años,Fac. Admon,Universitrario


In [79]:
demograficos_df[demograficos_df["CODIGO"]=="53212191201"].sort_values(by="PERIODO_ACAENICO",ascending=False).head(3)

,PERIODO_ACAENICO,CODIGO,UNIDAD,CODIGOPROGRAMA,PROGRAMA,SEMESTRE,PROMEDIOSEMESTRE,GENERO,FECHANACIMIENTO,ESTRATO,CIUDADRESIDENCIA,DEPARTAMENTORESICEDENCIA,ESTADOCIVIL,PROMEDIOGENERAL,SITUACION_ACTUAL,CATEGORIA_ACTUAL,SITUACION_DEL_PERIODO,CATEGORIA_DEL_PERIODO,DISCAPACIDAD,TIPO_DISCAPACIDAD,FECHA_MATRICULA,IDCIUDADNACIMIENTO,CIUDADNACIMIENTO,IDDEPARTAMENTONACIMIENTO,DEPARTAMENTONACIMIENTO,SISBEN,NIVELSISBEN,CANTIDAD,PERIODO,EDAD2,EDAD,FACULTAD,NIVEL_FORMACION
41608,2022 - 2,53212191201,ESCUELA NACIONAL DEL DEPORTE,53212,DEPORTE,8,"4,2",Masculino,23-09-1995,1.0,PALMIRA,VALLE DEL CAUCA,Soltero,"4,1",EXCLUIDO NO RENOVACION DE MATRICULA,CONTINUIDAD ACADEMICA - EGRESADO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,18/07/22,17001.0,MANIZALES,17.0,CALDAS,1.0,4.0,1,2022-II,27,Edad entre 26 y 33 años,Fac. Deporte,Universitrario
37802,2022 - 1,53212191201,ESCUELA NACIONAL DEL DEPORTE,53212,DEPORTE,7,"4,3",Masculino,23-09-1995,1.0,PALMIRA,VALLE DEL CAUCA,Soltero,4,EXCLUIDO NO RENOVACION DE MATRICULA,CONTINUIDAD ACADEMICA - EGRESADO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,30/12/21,17001.0,MANIZALES,17.0,CALDAS,1.0,4.0,1,2022-I,26,Edad entre 26 y 33 años,Fac. Deporte,Universitrario
33963,2021 - 2,53212191201,ESCUELA NACIONAL DEL DEPORTE,53212,DEPORTE,6,"3,9",Masculino,23-09-1995,1.0,PALMIRA,VALLE DEL CAUCA,Soltero,4,EXCLUIDO NO RENOVACION DE MATRICULA,CONTINUIDAD ACADEMICA - EGRESADO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,13/07/21,17001.0,MANIZALES,17.0,CALDAS,1.0,4.0,1,2021-II,26,Edad entre 26 y 33 años,Fac. Deporte,Universitrario


Se observa que el codigo de estudiante se repite tantas veces como periodos académicos haya cursado y las variables de semestre, promediosemestre, promediogeneral y fecha_matricula son las correspondientes al periodo academico, las demas variables demograficas son las mismas para todos los periodos academicos. Por lo anterior, se tomara la informacion demografica correspondiente al ultimo periodo academico.

In [80]:
idx = demograficos_df.groupby('CODIGO')['PERIODO_ACAENICO'].idxmax()

In [81]:
max_periodo_rows = demograficos_df.loc[idx]

In [82]:
demograficos_df_uniq=max_periodo_rows.copy()

In [83]:
total_rows_uniq = demograficos_df_uniq.shape[0]
total_rows_uniq

10021

El estudio se realizara con la informacion demografica correspondiente al ultimo periodo academico de 10.021 estudiantes unicos.

*  ***Calculo de la variable objetivo (Desercion)*** : Si la variable denominada “situación actual” contiene cumple las siguientes condiciones “excluido cancelación semestre” o “excluido no renovación matricula” o “retiro definitivo” y la variable denominada “categoría actual” el valor es diferente de “continuidad académica - egresado” se considera “Deserción”.

In [84]:
def calculate_target(row):
    if (row['SITUACION_ACTUAL'] == 'EXCLUIDO NO RENOVACION DE MATRICULA' or row["SITUACION_ACTUAL"]=='RETIRO DEFINITIVO' or row["SITUACION_ACTUAL"]=='EXCLUIDO CANCELACION SEMESTRE' ) and  (row['CATEGORIA_ACTUAL'] != "CONTINUIDAD ACADEMICA - EGRESADO") :
        return 0
    else:
        return 1

In [85]:
demograficos_df_uniq["target"]=demograficos_df_uniq.apply(calculate_target, axis=1)

In [88]:
demograficos_df_uniq["target"].value_counts()

,count
target,
1,8760
0,1261


Se observa que 1.261 estudiantes de 10.021 desertan, es decir la tasa de desercion de la poblacion estudiada es de 12,6% valor que coincide con las cifras de desercion reportadas por la entidad.

***Formato de variables numericas y fechas:*** Creacion de la variable YEARNACIMIENTO, conversion a valor numerico del promedio general y de semestre.

In [89]:
demograficos_df_uniq["YEARNACIMIENTO"]=pd.to_datetime(demograficos_df_uniq["FECHANACIMIENTO"])
demograficos_df_uniq['YEARNACIMIENTO'] = demograficos_df_uniq['YEARNACIMIENTO'].dt.year
field="PROMEDIOGENERAL"
demograficos_df_uniq[field] = pd.to_numeric(demograficos_df_uniq[field].str.replace(',', '.'), errors='coerce')
field="PROMEDIOSEMESTRE"
demograficos_df_uniq[field] = pd.to_numeric(demograficos_df_uniq[field].str.replace(',', '.'), errors='coerce')

<ipython-input-89-0d1c1fa71a41>:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  demograficos_df_uniq["YEARNACIMIENTO"]=pd.to_datetime(demograficos_df_uniq["FECHANACIMIENTO"])
